# Execution Context in OpenDXA

This tutorial covers the execution context in OpenDXA, which is a crucial component that manages state, resources, and coordination between different layers of the framework.

## Learning Objectives

By the end of this tutorial, you will understand:

1. The structure and components of the execution context
2. How to manage state across different layers
3. How to handle resources and their lifecycle
4. How to coordinate between workflow, planning, and reasoning layers
5. Best practices for context management

## Prerequisites

- Basic understanding of OpenDXA's 3-layer architecture
- Familiarity with Python async/await syntax
- Understanding of basic resource management concepts

## 1. Understanding the Execution Context

The execution context (`ExecutionContext`) is the central component that manages:

- State across different layers (workflow, planning, reasoning)
- Resource allocation and lifecycle
- Results and data sharing between layers
- Global context and configuration

In [ ]:
from opendxa.execution import ExecutionContext
from opendxa.agent import AgentState, WorldState, ExecutionState
from opendxa.common.resource import LLMResource

# Create LLM resources for different layers
workflow_llm = LLMResource()
planning_llm = LLMResource()
reasoning_llm = LLMResource()

# Initialize the execution context
context = ExecutionContext(
    workflow_llm=workflow_llm,
    planning_llm=planning_llm,
    reasoning_llm=reasoning_llm,
    agent_state=AgentState(),
    world_state=WorldState(),
    execution_state=ExecutionState(),
    global_context={"version": "1.0"}
)

print("Execution Context Created:")
print(f"- Workflow LLM: {context.workflow_llm}")
print(f"- Planning LLM: {context.planning_llm}")
print(f"- Reasoning LLM: {context.reasoning_llm}")
print(f"- Global Context: {context.global_context}")

## 2. State Management

The execution context manages three types of state:

1. **Agent State**: Tracks the agent's internal state
2. **World State**: Maintains information about the external environment
3. **Execution State**: Tracks the progress of execution across layers

Let's see how these states are managed:

In [ ]:
from opendxa.execution import ExecutionNode, ExecutionNodeStatus

# Update execution state
context.execution_state.status = "RUNNING"
context.execution_state.current_node_id = "node1"
context.execution_state.visited_nodes.append("node1")

# Update agent state
context.agent_state.metadata["current_task"] = "process_data"
context.agent_state.metadata["start_time"] = "2024-03-20T10:00:00"

# Update world state
context.world_state.metadata["environment"] = "production"
context.world_state.metadata["available_resources"] = ["cpu", "memory", "gpu"]

# Print current states
print("Execution State:")
print(f"- Status: {context.execution_state.status}")
print(f"- Current Node: {context.execution_state.current_node_id}")
print(f"- Visited Nodes: {context.execution_state.visited_nodes}")

print("\nAgent State:")
print(f"- Current Task: {context.agent_state.metadata.get('current_task')}")
print(f"- Start Time: {context.agent_state.metadata.get('start_time')}")

print("\nWorld State:")
print(f"- Environment: {context.world_state.metadata.get('environment')}")
print(f"- Available Resources: {context.world_state.metadata.get('available_resources')}")

## 3. Resource Management

The execution context manages resources for different layers. Let's see how to work with resources:

In [ ]:
from opendxa.common.resource import BaseResource

# Create custom resources
class DatabaseResource(BaseResource):
    """Example database resource."""
    def __init__(self, connection_string: str):
        super().__init__()
        self.connection_string = connection_string
        
    async def initialize(self) -> None:
        """Initialize database connection."""
        print(f"Connecting to database: {self.connection_string}")
        
    async def cleanup(self) -> None:
        """Cleanup database connection."""
        print("Closing database connection")

# Add resources to context
context.resources = {
    "database": DatabaseResource("postgresql://localhost:5432/mydb"),
    "workflow_llm": workflow_llm,
    "planning_llm": planning_llm,
    "reasoning_llm": reasoning_llm
}

# Initialize resources
for resource in context.resources.values():
    await resource.initialize()

print("Resources initialized:")
for name, resource in context.resources.items():
    print(f"- {name}: {type(resource).__name__}")

## 4. Results Management

The execution context maintains results from different layers. Let's see how to manage results:

In [ ]:
# Update workflow results
context.update_workflow_result(
    "workflow_node_1",
    {"status": "completed", "output": "processed data"}
)

# Update plan results
context.update_plan_result(
    "workflow_1",
    "plan_1",
    {"steps_completed": 3, "total_steps": 5}
)

# Update reasoning results
context.update_reasoning_result(
    "workflow_1",
    "plan_1",
    "reasoning_1",
    {"confidence": 0.95, "explanation": "High confidence in decision"}
)

# Retrieve results
print("Workflow Results:")
print(context.get_workflow_result("workflow_node_1"))

print("\nPlan Results:")
print(context.get_plan_results_for_workflow("workflow_1"))

print("\nReasoning Results:")
print(context.get_reasoning_results_for_plan("workflow_1", "plan_1"))

## 5. Building LLM Context

The execution context provides a method to build context for LLM calls, which includes current state and results:

In [ ]:
# Create example nodes
workflow_node = ExecutionNode(
    node_id="workflow_1",
    node_type="task",
    description="Process data"
)

plan_node = ExecutionNode(
    node_id="plan_1",
    node_type="task",
    description="Optimize process"
)

reasoning_node = ExecutionNode(
    node_id="reasoning_1",
    node_type="task",
    description="Analyze results"
)

# Set current nodes in context
context.current_workflow = workflow_node
context.current_plan = plan_node
context.current_reasoning = reasoning_node

# Build LLM context
llm_context = context.build_llm_context()

print("LLM Context:")
print("Global Context:")
print(llm_context["global_context"])
print("\nCurrent Nodes:")
print(f"- Workflow: {llm_context['current']['workflow']}")
print(f"- Plan: {llm_context['current']['plan']}")
print(f"- Reasoning: {llm_context['current']['reasoning']}")
print("\nResults:")
print(f"- Workflow: {llm_context['results']['workflow']}")
print(f"- Plan: {llm_context['results']['plan']}")
print(f"- Reasoning: {llm_context['results']['reasoning']}")

## 6. Best Practices

Here are some best practices for working with the execution context:

1. **Resource Management**
   - Always initialize resources before use
   - Clean up resources when done
   - Use appropriate resource types for each layer

2. **State Management**
   - Keep state updates atomic
   - Use appropriate state types for different concerns
   - Maintain clear state transitions

3. **Results Management**
   - Store results with appropriate metadata
   - Use consistent result formats
   - Clean up old results when no longer needed

4. **Context Building**
   - Keep global context minimal and relevant
   - Update current nodes appropriately
   - Build LLM context only when needed

## Summary

In this tutorial, we covered:

1. The structure and components of the execution context
2. How to manage state across different layers
3. How to handle resources and their lifecycle
4. How to manage results from different layers
5. How to build context for LLM calls
6. Best practices for working with the execution context

The execution context is a crucial component that enables coordination between different layers of the OpenDXA framework while maintaining state and managing resources effectively.